# Projet Final - Évaluation d'une Option Asiatique

**Université Paris Dauphine - Master 1  
Méthodes Monte-Carlo (2023-2024)**

## Contexte

On considère un actif risqué $S$ défini par
$$
S_t = S_0 \exp\Bigl(\Bigl(r - \frac{\sigma^2}{2}\Bigr)t + \sigma W_t\Bigr),
$$
avec $S_0 = 1$, $r = 0$, et $W_t$ un mouvement brownien standard.

On définit une grille de temps
$$
t_k = \frac{T}{n} k,\quad k = 0,1,\dots,n,
$$
avec $T = 1$. La moyenne arithmétique des prix est
$$
A_T^n = \frac{1}{n}\sum_{i=1}^{n} S_{t_i}.
$$

L'option asiatique a pour payoff
$$
G = \bigl(A_T^n - K\bigr)^+,
$$
avec $K = 1$.  
L'objectif est de calculer
$$
E[G] = E\Bigl[\bigl(A_T^n - K\bigr)^+\Bigr]
$$
par la méthode de Monte Carlo.

## Tâches à réaliser

1. **Simulation et estimation par Monte Carlo**  
   - Simuler les trajectoires du mouvement brownien sur la grille discrète.  
   - Simuler les trajectoires de $S_t$.  
   - Estimer $E[G]$.

2. **Méthode antithétique**  
   - Utiliser des variables antithétiques afin de réduire la variance.

3. **Variables de contrôle**  
   - Utiliser la variable de contrôle
     $$
     \Biggl(\Bigl(\prod_{i=1}^{n} S_{t_i}\Bigr)^{1/n} - K\Biggr)^+,
     $$
     pour améliorer l'estimation.

4. **Échantillonnage préférentiel (Importance Sampling)**  
   - Écrire $G = g(Z)$ avec $Z \sim \mathcal{N}(0,I_n)$.  
   - Montrer que, pour tout vecteur $\mu\in\mathbb{R}^n$,
     $$
     E[G] = E\Bigl[g(Z+\mu)\exp\Bigl(-\langle \mu,Z\rangle-\frac{\|\mu\|^2}{2}\Bigr)\Bigr].
     $$
   - En suivant [1], déterminer un vecteur $\hat{\mu}$ approché via la méthode de bifurcation.

5. **(Bonus) Stratification**  
   - Proposer une méthode de stratification pour réduire encore la variance.

La cellule de code suivante présente une implémentation en Python de ces différentes méthodes.


In [1]:
# Importation des bibliothèques nécessaires
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.optimize import bisect

# Paramètres de l'option et du modèle
S0 = 1.0      # Prix initial
r = 0.0       # Taux d'intérêt (nul)
sigma = 0.25  # Volatilité
T = 1.0       # Maturité
K = 1.0       # Strike
n = 50        # Nombre de pas de temps
dt = T / n    # Pas de temps
M = 10000     # Nombre de simulations Monte Carlo

#####################################
# 1. Simulation et Monte Carlo Standard
#####################################
def simulate_asset_paths(M, n, S0, r, sigma, dt, antithetic=False):
    """
    Simule les trajectoires de S_t sur la grille discrète.
    Si antithetic=True, utilise la méthode des variables antithétiques.
    Renvoie un tableau de dimension (M, n+1) contenant les trajectoires.
    """
    if not antithetic:
        # Simulation classique
        dW = np.sqrt(dt) * np.random.randn(M, n)
        S = np.zeros((M, n+1))
        S[:, 0] = S0
        for i in range(n):
            S[:, i+1] = S[:, i] * np.exp((r - 0.5 * sigma**2) * dt + sigma * dW[:, i])
        return S
    else:
        # Simulation par variables antithétiques : M/2 paires
        M_half = M // 2
        dW = np.sqrt(dt) * np.random.randn(M_half, n)
        dW_anti = -dW  # trajectoires opposées
        S1 = np.zeros((M_half, n+1))
        S2 = np.zeros((M_half, n+1))
        S1[:, 0] = S0
        S2[:, 0] = S0
        for i in range(n):
            S1[:, i+1] = S1[:, i] * np.exp((r - 0.5 * sigma**2) * dt + sigma * dW[:, i])
            S2[:, i+1] = S2[:, i] * np.exp((r - 0.5 * sigma**2) * dt + sigma * dW_anti[:, i])
        S = np.concatenate([S1, S2], axis=0)
        return S

def asian_option_payoff(S, K):
    """
    Calcule le payoff de l'option asiatique pour chaque trajectoire.
    La moyenne est calculée sur les instants t_1 à t_n.
    """
    A = np.mean(S[:, 1:], axis=1)
    payoff = np.maximum(A - K, 0)
    return payoff

def monte_carlo_price(S, K):
    """
    Estime le prix de l'option par la méthode de Monte Carlo.
    Renvoie la moyenne des payoffs et l'erreur-type.
    """
    payoff = asian_option_payoff(S, K)
    price = np.mean(payoff)
    std_error = np.std(payoff) / np.sqrt(len(payoff))
    return price, std_error

# Simulation classique
S_paths = simulate_asset_paths(M, n, S0, r, sigma, dt, antithetic=False)
price_mc, se_mc = monte_carlo_price(S_paths, K)
print("Monte Carlo Standard:")
print("Prix: {:.4f}, Erreur-type: {:.4f}".format(price_mc, se_mc))

#####################################
# 2. Variance Reduction par Variables Antithétiques
#####################################
S_paths_anti = simulate_asset_paths(M, n, S0, r, sigma, dt, antithetic=True)
price_anti, se_anti = monte_carlo_price(S_paths_anti, K)
print("\nMéthode Antithétique:")
print("Prix: {:.4f}, Erreur-type: {:.4f}".format(price_anti, se_anti))

#####################################
# 3. Réduction de Variance par Variables de Contrôle
#####################################
def control_variate_payoff(S, K):
    """
    Calcule le payoff de la variable de contrôle basé sur la moyenne géométrique:
    $$\Bigl((\prod_{i=1}^{n} S_{t_i})^{1/n} - K\Bigr)^+.$$
    """
    # Moyenne géométrique: exp( moyenne des logarithmes )
    geo_mean = np.exp(np.mean(np.log(S[:, 1:]), axis=1))
    payoff_cv = np.maximum(geo_mean - K, 0)
    return payoff_cv

# Calcul des payoffs
payoff = asian_option_payoff(S_paths, K)
control_payoff = control_variate_payoff(S_paths, K)

# Prix exact (fermé) de l'option asiatique géométrique (approximation)
def geometric_asian_option_price(S0, K, r, sigma, T, n):
    """
    Calcule le prix de l'option asiatique géométrique en formule fermée.
    Voir la littérature pour la formule exacte.
    """
    dt = T/n
    sigma_sq_geo = sigma**2 * ((n+1)*(2*n+1)) / (6*n**2)
    sigma_geo = np.sqrt(sigma_sq_geo)
    mu_geo = (r - 0.5*sigma**2)*(T + dt)/2 + 0.5*sigma_sq_geo
    d1 = (np.log(S0/K) + mu_geo) / sigma_geo
    d2 = d1 - sigma_geo
    # Ici r = 0 donc le facteur d'actualisation vaut 1
    price_geo = S0 * np.exp(mu_geo) * norm.cdf(d1) - K * norm.cdf(d2)
    return price_geo

price_geo_exact = geometric_asian_option_price(S0, K, r, sigma, T, n)

# Estimation par variables de contrôle
cov_matrix = np.cov(payoff, control_payoff)
beta = - cov_matrix[0,1] / cov_matrix[1,1]
adjusted_payoff = payoff + beta * (control_payoff - price_geo_exact)
price_cv = np.mean(adjusted_payoff)
se_cv = np.std(adjusted_payoff) / np.sqrt(len(adjusted_payoff))
print("\nVariables de Contrôle:")
print("Prix: {:.4f}, Erreur-type: {:.4f}".format(price_cv, se_cv))

#####################################
# 4. Échantillonnage Préférentiel (Importance Sampling)
#####################################
# On souhaite exprimer le payoff comme fonction de Z = (Z_1,...,Z_n) ~ N(0,I_n)
# La dynamique est : 
# S_{t_{i+1}} = S_{t_i} exp((r - sigma^2/2)*dt + sigma*sqrt(dt)*Z_{i+1})

# Définition d'une fonction pour déterminer, via bifurcation, le paramètre y tel que
# $$ \frac{1}{n}\sum_{j=1}^{n} S_j(y) - K - y = 0. $$
def compute_mu_hat(y, S0, sigma, dt, n, K):
    """
    Pour un y donné, calcule la séquence z(y) et S(y) et renvoie
    $$ \frac{1}{n}\sum_{j=1}^{n} S_j(y) - K - y. $$
    """
    z = np.zeros(n)
    S_vec = np.zeros(n)
    # Calcul de z_1(y)
    z[0] = sigma * np.sqrt(dt) * (y + K) / y
    t1 = dt
    S_vec[0] = S0 * np.exp(-0.5*sigma**2 * t1 + sigma * np.sqrt(dt) * z[0])
    for j in range(1, n):
        z[j] = z[j-1] - sigma * np.sqrt(dt) * S_vec[j-1] / (n * y)
        t_j = (j+1) * dt
        S_vec[j] = S0 * np.exp(-0.5*sigma**2 * t_j + sigma * np.sqrt(dt) * np.sum(z[:j+1]))
    avg_S = np.mean(S_vec)
    return avg_S - K - y

# Recherche de y_hat par la méthode de bissection
y_lower = 1e-4
y_upper = 1.0
while compute_mu_hat(y_lower, S0, sigma, dt, n, K) * compute_mu_hat(y_upper, S0, sigma, dt, n, K) > 0:
    y_upper *= 2

y_hat = bisect(lambda y: compute_mu_hat(y, S0, sigma, dt, n, K), y_lower, y_upper, xtol=1e-4)
print("\nÉchantillonnage Préférentiel:")
print("y_hat =", y_hat)

# À partir de y_hat, on déduit mu_hat = z(y_hat)
def compute_mu_from_y(y, S0, sigma, dt, n):
    z = np.zeros(n)
    z[0] = sigma * np.sqrt(dt) * (y + K) / y
    for j in range(1, n):
        # On reconstruit S_j avec les z calculés jusqu'ici
        S_j = S0 * np.exp(-0.5*sigma**2 * (j*dt) + sigma * np.sqrt(dt) * np.sum(z[:j]))
        z[j] = z[j-1] - sigma * np.sqrt(dt) * S_j / (n * y)
    return z

mu_hat = compute_mu_from_y(y_hat, S0, sigma, dt, n)
print("mu_hat =", mu_hat)

# Simulation par Importance Sampling avec décalage mu_hat
def simulate_paths_importance_sampling(M, n, S0, r, sigma, dt, mu):
    """
    Simule les trajectoires avec un décalage mu dans le vecteur Z.
    Sous la nouvelle mesure, on a Z = Z' + mu, avec Z' ~ N(0,1).
    """
    # On simule Z' puis on ajoute mu
    Z_shifted = np.random.randn(M, n) + mu  # chaque trajectoire est décalée
    # Reconstruction des trajectoires de S_t
    S = np.zeros((M, n+1))
    S[:, 0] = S0
    for i in range(n):
        S[:, i+1] = S[:, i] * np.exp((r - 0.5 * sigma**2)*dt + sigma * np.sqrt(dt) * Z_shifted[:, i])
    # On récupère Z' = Z_shifted - mu pour le calcul du facteur de vraisemblance
    Z_original = Z_shifted - mu
    return S, Z_original

S_paths_is, Z_samples = simulate_paths_importance_sampling(M, n, S0, r, sigma, dt, mu_hat)
# Calcul du facteur de vraisemblance:
# $$ \exp\Bigl(-\langle \mu, Z\rangle - \frac{\|\mu\|^2}{2}\Bigr) $$
weights = np.exp(-np.sum(mu_hat * Z_samples, axis=1) - 0.5 * np.sum(mu_hat**2))
payoff_is = asian_option_payoff(S_paths_is, K)
weighted_payoff = payoff_is * weights
price_is = np.mean(weighted_payoff)
se_is = np.std(weighted_payoff) / np.sqrt(M)
print("Prix par Importance Sampling: {:.4f}, Erreur-type: {:.4f}".format(price_is, se_is))

#####################################
# 5. (Bonus) Stratification
#####################################
def stratified_sampling(M, n, S0, r, sigma, dt, K, strata=10):
    """
    Implémente une stratification simple sur la première variable Z_1.
    Pour chaque strate, on fixe la valeur médiane du sous-intervalle de N(0,1).
    """
    M_per_stratum = M // strata
    S_all = []
    for i in range(strata):
        # Définition de la strate pour Z_1
        a = norm.ppf(i / strata)
        b = norm.ppf((i+1) / strata)
        Z1_val = norm.ppf((i + 0.5) / strata)  # valeur médiane de la strate
        # Simulation des autres pas
        dW_remaining = np.sqrt(dt) * np.random.randn(M_per_stratum, n-1)
        dW_full = np.hstack([Z1_val * np.ones((M_per_stratum, 1)), dW_remaining])
        S = np.zeros((M_per_stratum, n+1))
        S[:, 0] = S0
        for j in range(n):
            S[:, j+1] = S[:, j] * np.exp((r - 0.5 * sigma**2)*dt + sigma * dW_full[:, j])
        S_all.append(S)
    S_all = np.vstack(S_all)
    payoff_all = asian_option_payoff(S_all, K)
    price_strat = np.mean(payoff_all)
    se_strat = np.std(payoff_all) / np.sqrt(len(payoff_all))
    return price_strat, se_strat

price_strat, se_strat = stratified_sampling(M, n, S0, r, sigma, dt, K, strata=10)
print("\nStratification:")
print("Prix: {:.4f}, Erreur-type: {:.4f}".format(price_strat, se_strat))


<>:86: SyntaxWarning: invalid escape sequence '\B'
<>:136: SyntaxWarning: invalid escape sequence '\s'
<>:86: SyntaxWarning: invalid escape sequence '\B'
<>:136: SyntaxWarning: invalid escape sequence '\s'
C:\Users\ugo11\AppData\Local\Temp\ipykernel_8260\2721999679.py:86: SyntaxWarning: invalid escape sequence '\B'
  """
C:\Users\ugo11\AppData\Local\Temp\ipykernel_8260\2721999679.py:136: SyntaxWarning: invalid escape sequence '\s'
  """


Monte Carlo Standard:
Prix: 0.0585, Erreur-type: 0.0009

Méthode Antithétique:
Prix: 0.0587, Erreur-type: 0.0010

Variables de Contrôle:
Prix: 0.0582, Erreur-type: 0.0000

Échantillonnage Préférentiel:
y_hat = 0.15834833374023435
mu_hat = [0.25863106 0.25412735 0.24958582 0.2450069  0.24039104 0.23573868
 0.23105032 0.22632645 0.22156761 0.21677434 0.21194721 0.20708682
 0.20219376 0.19726868 0.19231223 0.18732508 0.18230793 0.17726149
 0.1721865  0.16708371 0.16195389 0.15679784 0.15161636 0.1464103
 0.14118048 0.13592777 0.13065307 0.12535724 0.12004122 0.11470593
 0.1093523  0.10398129 0.09859387 0.09319101 0.08777371 0.08234297
 0.0768998  0.07144522 0.06598025 0.06050595 0.05502336 0.04953351
 0.04403749 0.03853633 0.03303112 0.02752292 0.02201281 0.01650185
 0.01099111 0.00548168]
Prix par Importance Sampling: 0.0586, Erreur-type: 0.0003

Stratification:
Prix: 0.1273, Erreur-type: 0.0020
